# Cleaning the data so it can be used for machine learning models

In [1]:
import pandas as pd
import numpy as np

## Read in the dataset

In [2]:
movies_metadata = pd.read_csv("../data/movies-metadata.csv", thousands=",").drop(columns=['Unnamed: 0', '_id'])
movies_metadata.shape

(8873, 19)

## Eliminate Series (non-movies)

find how many "movies" are acutally series

In [3]:
movies_metadata.loc[movies_metadata['Type'] == 'series'].shape

(59, 19)

eliminate these series from the DataFrame

In [4]:
movies_metadata_cleaned = movies_metadata.drop(movies_metadata.loc[movies_metadata['Type'] == 'series'].index)
movies_metadata_cleaned.shape

(8814, 19)

## Handle missing values and duplicates

In [5]:
movies_metadata_cleaned = movies_metadata_cleaned.drop_duplicates()

In [6]:
movies_metadata_cleaned.isnull().sum()

Language        70
Box_office    2729
Country         22
Rated         1507
imdbID           0
Awards        2755
Poster         639
Director       118
Released       241
Writer         634
imdbVotes      346
Runtime        236
Response         0
imdbRating     345
Title            0
Genre           31
Actors         158
Type             0
Metascore     3997
dtype: int64

### Let's first eliminate all movies without a movie poster

In [7]:
movies_metadata_cleaned = movies_metadata_cleaned.dropna(subset=['Poster'])
movies_metadata_cleaned.shape

(8163, 19)

In [8]:
movies_metadata_cleaned.isnull().sum()

Language        23
Box_office    2124
Country          5
Rated          946
imdbID           0
Awards        2190
Poster           0
Director         9
Released        56
Writer         365
imdbVotes       32
Runtime         64
Response         0
imdbRating      31
Title            0
Genre           11
Actors          58
Type             0
Metascore     3375
dtype: int64

we will fill missing values and check the remaining missing values as we go

fill missing genres with N/A

In [9]:
movies_metadata_cleaned['Genre'].fillna("N/A", inplace=True)

In [10]:
movies_metadata_cleaned['Rated'].value_counts()

R            3358
PG-13        2025
PG           1254
NOT RATED     260
G             170
UNRATED       119
NC-17          15
X               7
TV-MA           3
Not Rated       1
M/PG            1
TV-14           1
PASSED          1
AO              1
TV-PG           1
Name: Rated, dtype: int64

We will also drop any movie that isn't either g, pg, pg-13, r, or not rated 

In [11]:
# convert to upper case bc of the one "not rated" value with different cases 
movies_metadata_cleaned['Rated'] = movies_metadata_cleaned['Rated'].str.upper()
movies_metadata_cleaned = movies_metadata_cleaned.loc[movies_metadata_cleaned['Rated'].isin(['R', 'PG-13', 'PG', 'G', 'NOT RATED', 'UNRATED'])]
movies_metadata_cleaned['Rated'].value_counts()

R            3358
PG-13        2025
PG           1254
NOT RATED     261
G             170
UNRATED       119
Name: Rated, dtype: int64

In [12]:
movies_metadata_cleaned.isnull().sum()

Language         8
Box_office    1688
Country          2
Rated            0
imdbID           0
Awards        1770
Poster           0
Director         2
Released        19
Writer         191
imdbVotes        1
Runtime         15
Response         0
imdbRating       0
Title            0
Genre            0
Actors          14
Type             0
Metascore     2754
dtype: int64

Fill language, country, and runtime with the modes in their respective column

In [13]:
for col in ['Language', 'Country', 'Runtime']:
    movies_metadata_cleaned[col].fillna(movies_metadata_cleaned[col].mode()[0], inplace=True)

In [14]:
movies_metadata_cleaned.isnull().sum()

Language         0
Box_office    1688
Country          0
Rated            0
imdbID           0
Awards        1770
Poster           0
Director         2
Released        19
Writer         191
imdbVotes        1
Runtime          0
Response         0
imdbRating       0
Title            0
Genre            0
Actors          14
Type             0
Metascore     2754
dtype: int64

replace the one movie without a value for imdbVotes with the average

In [15]:
movies_metadata_cleaned[movies_metadata_cleaned['imdbVotes'].isnull()]

,Language,Box_office,Country,Rated,imdbID,Awards,Poster,Director,Released,Writer,imdbVotes,Runtime,Response,imdbRating,Title,Genre,Actors,Type,Metascore
523,English,4671831.0,USA,R,tt0088258,3 wins & 2 nominations.,http://ia.media-imdb.com/images/M/MV5BMTQ2MTIz...,Rob Reiner,02 Mar 1984,"Christopher Guest, Michael McKean, Harry Shear...",NaN,82 min,True,8.0,This Is Spinal Tap,"Comedy, Music","Rob Reiner, Kimberly Stringer, Chazz Dominguez...",movie,85.0


In [16]:
movies_metadata_cleaned['imdbVotes'].fillna(movies_metadata_cleaned['imdbVotes'].mean(), inplace=True)

"The Animation SHow" is a collection of short films. It can be eliminated.

In [17]:
movies_metadata_cleaned[movies_metadata_cleaned['Director'].isnull()]

,Language,Box_office,Country,Rated,imdbID,Awards,Poster,Director,Released,Writer,imdbVotes,Runtime,Response,imdbRating,Title,Genre,Actors,Type,Metascore
4450,"English, Japanese, German",NaN,USA,R,tt0372763,NaN,http://ia.media-imdb.com/images/M/MV5BMTI3MzE1...,NaN,18 Jul 2003,NaN,469.0,94 min,True,7.6,The Animation Show,"Animation, Comedy",NaN,movie,NaN
5090,English,NaN,USA,NOT RATED,tt0436070,NaN,http://ia.media-imdb.com/images/M/MV5BMTI3MzE1...,NaN,18 Feb 2005,NaN,201.0,85 min,True,7.1,The Animation Show 2005,Animation,NaN,movie,76.0


In [18]:
movies_metadata_cleaned = movies_metadata_cleaned.dropna(subset=['Director'])

In [19]:
movies_metadata_cleaned.isnull().sum()

Language         0
Box_office    1686
Country          0
Rated            0
imdbID           0
Awards        1768
Poster           0
Director         0
Released        19
Writer         189
imdbVotes        0
Runtime          0
Response         0
imdbRating       0
Title            0
Genre            0
Actors          12
Type             0
Metascore     2753
dtype: int64

In [20]:
movies_metadata_cleaned[movies_metadata_cleaned['Released'].isnull()]

,Language,Box_office,Country,Rated,imdbID,Awards,Poster,Director,Released,Writer,imdbVotes,Runtime,Response,imdbRating,Title,Genre,Actors,Type,Metascore
298,"English, German, French",NaN,USA,PG,tt0083947,NaN,http://ia.media-imdb.com/images/M/MV5BODgyNDI3...,Fred Zinnemann,NaN,"Michael Austin, Kay Boyle (story)",725.0,108 min,True,6.1,Five Days One Summer,Drama,"Sean Connery, Betsy Brantley, Lambert Wilson, ...",movie,NaN
1281,English,NaN,USA,PG,tt0095047,1 nomination.,http://ia.media-imdb.com/images/M/MV5BMTE5OTgx...,Charles Matthau,NaN,"Darren Star (story), Andrew Licht (story), Jef...",192.0,85 min,True,4.6,Doin' Time on Planet Earth,Comedy,"Nicholas Strouse, Andrea Thompson, Martha Scot...",movie,NaN
2825,English,NaN,USA,R,tt0117638,NaN,http://ia.media-imdb.com/images/M/MV5BMTQ0MDgy...,Dan Bell,NaN,Dan Bell,36.0,83 min,True,5.6,The Shot,Comedy,"Dan Bell, Michael Rivkin, Jude Horowitz, Vince...",movie,NaN
3209,English,1025234.0,USA,R,tt0127722,3 wins & 1 nomination.,http://ia.media-imdb.com/images/M/MV5BMTYwMjA1...,Larry Clark,NaN,"Eddie Little (book), Christopher Landon, Steph...",7091.0,101 min,True,6.5,Another Day in Paradise,"Crime, Drama, Thriller","James Woods, Melanie Griffith, Vincent Karthei...",movie,NaN
3244,"English, Serbo-Croatian, French",12903.0,USA,R,tt0120070,2 wins.,http://ia.media-imdb.com/images/M/MV5BMTkwNTE1...,Predrag Antonijevic,NaN,Robert Orr,8529.0,103 min,True,7.3,Savior,"Drama, War","Nastassja Kinski, Pascal Rollin, Catlin Foster...",movie,NaN
3464,English,79823.0,USA,R,tt0220095,NaN,http://ia.media-imdb.com/images/M/MV5BMTA2OTc3...,Richard Cummings Jr.,NaN,"Ernest Nyle Brown, Julie Shannon (story)",505.0,91 min,True,4.9,Thicker Than Water,Drama,"Mack 10, Fat Joe, Ice Cube, MC Eiht",movie,NaN
4191,English,NaN,USA,PG-13,tt0259017,NaN,http://ia.media-imdb.com/images/M/MV5BMTc5NDEz...,Robert J. Emery,NaN,"Robert J. Emery, Don Robertson (novel)",163.0,105 min,True,6.6,Swimming Upstream,Drama,"Matt Czuchry, Ben Savage, Jeffrey DeMunn, Kell...",movie,NaN
4487,English,NaN,Canada,R,tt0351708,NaN,http://ia.media-imdb.com/images/M/MV5BOTYzMzY2...,Eduardo Arnal,NaN,Eduardo Arnal,42.0,30 min,True,7.9,The Sea,Short,"Stefany Mathias, Eric Blais, Andrew Jackson, P...",movie,NaN
4505,English,NaN,USA,NOT RATED,tt0338497,NaN,http://ia.media-imdb.com/images/M/MV5BMTM5NzE3...,Danny Provenzano,NaN,"Ted A. Bohus, Danny Provenzano",501.0,100 min,True,5.4,This Thing of Ours,"Crime, Drama","Frank Vincent, Vincent Pastore, Louis Vanaria,...",movie,36.0
5422,English,252726.0,USA,PG,tt0785025,NaN,http://ia.media-imdb.com/images/M/MV5BMTc1MTM2...,Michael Landon Jr.,NaN,"Michael Landon Jr. (teleplay), Douglas Lloyd M...",1222.0,87 min,True,7.2,Love's Abiding Joy,"Drama, Western","Erin Cottrell, Dale Midkiff, Logan Bartholomew...",movie,NaN


Replace the `Released` with Wikipedia Release date. Otherwise, remove it

In [21]:
movies_metadata_cleaned.loc[movies_metadata_cleaned['imdbID'] == "tt0083947", "Released"] = "12 Nov 1982"
movies_metadata_cleaned.loc[movies_metadata_cleaned['imdbID'] == "tt0095047", 'Released'] = "16 Sep 1988"
movies_metadata_cleaned.loc[movies_metadata_cleaned['imdbID'] == "tt0127722", 'Released'] = "30 Dec 1998"
movies_metadata_cleaned.loc[movies_metadata_cleaned['imdbID'] == "tt0120070", 'Released'] = "20 Nov 1998"
movies_metadata_cleaned.loc[movies_metadata_cleaned['imdbID'] == "tt0220095", 'Released'] = "20 Oct 1999"
movies_metadata_cleaned.loc[movies_metadata_cleaned['imdbID'] == "tt0338497", 'Released'] = "18 Jul 2003"
movies_metadata_cleaned.loc[movies_metadata_cleaned['imdbID'] == "tt0785025", 'Released'] = "06 Oct 2006"
movies_metadata_cleaned.loc[movies_metadata_cleaned['imdbID'] == "tt0418206", 'Released'] = "21 Aug 2007"
movies_metadata_cleaned.loc[movies_metadata_cleaned['imdbID'] == "tt0838241", 'Released'] = "12 Jun 2009"
movies_metadata_cleaned.loc[movies_metadata_cleaned['imdbID'] == "tt1326900", 'Released'] = "01 Jul 2009"
movies_metadata_cleaned.loc[movies_metadata_cleaned['imdbID'] == "tt1606259", 'Released'] = "21 Jan 2011"
movies_metadata_cleaned.loc[movies_metadata_cleaned['imdbID'] == "tt1634003", 'Released'] = "27 Aug 2014"

In [22]:
movies_metadata_cleaned = movies_metadata_cleaned.dropna(subset=['Released'])

In [23]:
movies_metadata_cleaned.isnull().sum()

Language         0
Box_office    1680
Country          0
Rated            0
imdbID           0
Awards        1761
Poster           0
Director         0
Released         0
Writer         189
imdbVotes        0
Runtime          0
Response         0
imdbRating       0
Title            0
Genre            0
Actors          12
Type             0
Metascore     2747
dtype: int64

Fill actors and writer with some string specifying the unknown

In [24]:
for col in ['Writer', 'Actors']:
    movies_metadata_cleaned[col].fillna('Unknown', inplace=True)

In [25]:
movies_metadata_cleaned.isnull().sum()

Language         0
Box_office    1680
Country          0
Rated            0
imdbID           0
Awards        1761
Poster           0
Director         0
Released         0
Writer           0
imdbVotes        0
Runtime          0
Response         0
imdbRating       0
Title            0
Genre            0
Actors           0
Type             0
Metascore     2747
dtype: int64

Fill awards with different numerical encoding

In [26]:
movies_metadata_cleaned['Oscar_noms'] = 0
movies_metadata_cleaned['Oscar_wins'] = 0
movies_metadata_cleaned['Golden_globe_noms'] = 0
movies_metadata_cleaned['Golden_globe_wins'] = 0
movies_metadata_cleaned['BAFTA_noms'] = 0
movies_metadata_cleaned['BAFTA_wins'] = 0
movies_metadata_cleaned['Other_noms'] = 0
movies_metadata_cleaned['Other_wins'] = 0

In [27]:
awards_array = movies_metadata_cleaned['Awards'].values
awards_array[:30]

array(['Won 1 Oscar. Another 15 wins & 18 nominations.', '1 nomination.',
       'Nominated for 1 Golden Globe. Another 2 wins & 6 nominations.',
       '1 win.', 'Nominated for 3 Oscars. Another 1 win & 3 nominations.',
       'Won 1 Oscar. Another 8 wins & 13 nominations.',
       '1 win & 4 nominations.',
       'Nominated for 1 Oscar. Another 3 wins & 7 nominations.', '1 win.',
       'Won 4 Oscars. Another 15 wins & 14 nominations.',
       '3 wins & 4 nominations.',
       'Nominated for 2 Golden Globes. Another 4 nominations.',
       '3 wins & 5 nominations.', '1 nomination.', '1 nomination.',
       '1 nomination.', 'Nominated for 1 Oscar. Another 1 win.',
       '1 nomination.',
       'Nominated for 3 Golden Globes. Another 3 wins & 7 nominations.',
       'Nominated for 2 BAFTA Film Awards. Another 2 wins & 9 nominations.',
       'Nominated for 8 Oscars. Another 10 wins & 14 nominations.',
       '1 nomination.', 'Won 2 Oscars. Another 23 wins & 24 nominations.',
       '1

In [28]:
import re # import regexp
def get_awards(series_object):
    # if series_object['Awards'].isnull():
    #     return
    if isinstance(series_object['Awards'], str):
        awards = series_object['Awards'].split(". ")
        for award in awards:
            award_lower = award.lower()
            numeric = re.sub("[^0-9]", " ", award).split()
            if "Oscar" in award_lower:
                if "nominated" in award_lower:
                    series_object['Oscar_noms'] = int(numeric[0])
                else:
                    series_object['Oscar_wins'] = int(numeric[0])
            
            elif "golden globe" in award_lower:
                if "nominated" in award_lower:
                    series_object['Golden_globe_noms'] = int(numeric[0])
                else:
                    series_object['Golden_globe_wins'] = int(numeric[0])

            elif "bafta" in award_lower:
                if "nominated" in award_lower:
                    series_object['BAFTA_noms'] = int(numeric[0])
                else:
                    series_object['BAFTA_wins'] = int(numeric[0])
            
            elif "&" in award_lower:
                wins, noms = award_lower.split("&")
                series_object['Other_wins'] = int(re.sub("[^0-9]", " ", wins))
                series_object['Other_noms'] = int(re.sub("[^0-9]", " ", noms))
            
            elif "win" in award_lower:
                series_object['Other_wins'] = int(numeric[0])
            
            elif "nomination" in award_lower:
                series_object['Other_noms'] = int(numeric[0])
    return series_object

In [29]:
movies_metadata_cleaned = movies_metadata_cleaned.apply(get_awards, axis=1)

In [30]:
movies_metadata_cleaned = movies_metadata_cleaned.drop(columns=['Awards', 'Response', 'Type'])

In [31]:
movies_metadata_cleaned.isnull().sum()

Language                0
Box_office           1680
Country                 0
Rated                   0
imdbID                  0
Poster                  0
Director                0
Released                0
Writer                  0
imdbVotes               0
Runtime                 0
imdbRating              0
Title                   0
Genre                   0
Actors                  0
Metascore            2747
Oscar_noms              0
Oscar_wins              0
Golden_globe_noms       0
Golden_globe_wins       0
BAFTA_noms              0
BAFTA_wins              0
Other_noms              0
Other_wins              0
dtype: int64

In [32]:
print(movies_metadata_cleaned['Metascore'].median())
print(movies_metadata_cleaned['Metascore'].mean())

55.0
54.24960505529226


These values are similar, so we can confidently fill the remaining 2700+ values with the mean?

In [33]:
movies_metadata_cleaned['Metascore'].fillna(movies_metadata_cleaned['Metascore'].mean(), inplace=True)

In [34]:
movies_metadata_cleaned.isnull().sum()

Language                0
Box_office           1680
Country                 0
Rated                   0
imdbID                  0
Poster                  0
Director                0
Released                0
Writer                  0
imdbVotes               0
Runtime                 0
imdbRating              0
Title                   0
Genre                   0
Actors                  0
Metascore               0
Oscar_noms              0
Oscar_wins              0
Golden_globe_noms       0
Golden_globe_wins       0
BAFTA_noms              0
BAFTA_wins              0
Other_noms              0
Other_wins              0
dtype: int64

Now the only remaining missing column is `Box_office`

## Additional column for month

In [35]:
month_map = {
    'Jan': 0,
    'Feb': 1,
    'Mar': 2,
    'Apr': 3,
    'May': 4,
    'Jun': 5,
    'Jul': 6,
    'Aug': 7,
    'Sep': 8,
    'Oct': 9,
    'Nov': 10,
    'Dec': 11,
}
def create_month(series_object, month_map=month_map):
    month = month_map[series_object['Released'].split()[1]]
    return month

In [36]:
movies_metadata_cleaned['Release_month'] = movies_metadata_cleaned.apply(create_month, axis=1)

## Fill runtime with int

In [37]:
def runtime_int(series_object):
    return series_object['Runtime'].split()[0]

In [38]:
movies_metadata_cleaned['Runtime'] = movies_metadata_cleaned.apply(runtime_int, axis=1)

## Save the cleaned dataframe

In [39]:
movies_metadata_cleaned.to_csv("../data/movies-metadata-cleaned.csv", index=False)